# NOTEBOOK DE TRAITEMENT DES DONNEES DE RESSENTI 

In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import sys

sys.path.append(r"C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees")
sys.path.append(r"C:\Users\martin.schoreisz\git\Outils\Outils\src")
sys.path.append(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\Codes\src\Carac_situ_acoustiq"
)
sys.path.append(
    r"C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\donnees_produites\Codes\src"
)

import locale, os
from collections import Counter
from datetime import datetime, time, timedelta
from math import cos, degrees, log, log10, pi, pow, radians, sin

import altair as alt
import numpy as np
import pandas as pd
from Bruit.Emission import Route
from Bruit.Meteo import (
    DonneesMeteo,
    calculCategorieVentQualitatif,
    calculConditionPropagationMeteoFranceQualitatif,
    calculGradientVitesseSon,
    conditionPropagation,
    dfAngle,
    dicoSecteurAngulaire,
    graphCompTempMesureeHautBas,
    secteurAngulaireQuantitatif,
)
from Bruit.Niveaux import calculLden, calculTNI, niveau2Pression, sommeEnergetique, moyenneQuadratiquePression, pression2Niveau, calculHarmonica
from Expoitation_mesures.Acoustique import recupDonneesAgregees, recupFractileAgregees, recupDonneesAcoustiqueSiteInstru, recupHarmonica
from Expoitation_mesures.Analyses_croises import (
    creerCorrelation,
    fusionnerDfSources,
    graphCroiseBruitMeteo1Jour,
    graphCroiseBruitMeteoPlusieursJour,
    graphCroiseBruitMeteoTrafic,
    graphCroiseBruitTrafic6min,
)
from Expoitation_mesures.Meteo import (
    concatMeteoFranceEtSite,
    creerGraphCompConditionsPropa,
    creerGraphCompVentSiteMeteoFrance,
    creerGraphMeteo,
    recupDonneesMeteoBase,
    recupDonneesMeteofranceBase,
)
from Exploitation_ressenti.Analyse_declaration import (
    filtrerDeclas,
    passageDeclas6min,
    nbParticipantParJour,
    graphNbVotantParJour,
    grouperDeclaParHorodate,
    graphEvolutionMoyMinMaxStd,
    creerChartDeclaHeureMinute,
    ajouterIndicateursTest,
    graphMoyennesPonderees,
    ajouterMoyennePonderee,
    graphHarmonica,
    graphComparaisonBruitDecla,
    nbParticipantParPasTemp
)
from Connexions.Connexion_Transfert import ConnexionBdd
from Expoitation_mesures.Trafic import *
from Import_stockage_donnees.Import_export_bdd import ResultatsEnquete
from Import_stockage_donnees.Params import dicoInstruSite
from Outils.Outils import dateTexteDepuisDayOfYear
from math import log10, pow, radians, pi, cos, sin, degrees

locale.setlocale(locale.LC_ALL, "fr_FR")
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_colwidth", 100)
# alt.data_transformers.disable_max_rows()
from toolz.curried import pipe
def altairDataDir(data, data_dir='altairData'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('altairDataDir', altairDataDir)
alt.data_transformers.enable('altairDataDir')

DataTransformerRegistry.enable('altairDataDir')

# ANALYSE DES DECLARATIONS DE GENE
Il s'agit de fournir des indicateurs de répartition des différentes valeurs de contexte et de déclaration fournies par les participants. Attention, les formulaires papiers ne permettaient pas de remplir tout les informations de contexte

<div class="alert alert-block alert-warning"><b>Attention:</b> Il n'y a pas d'analyse des commentaires réalisée pour le moment.</div>

## analyse globale sur les déclarations

In [9]:
resultats = ResultatsEnquete()
resultats.creationCharts(export=False)
# si export = True, alors les données sont excportées dans le dossier dossierExportChartsRessenti du module Params
# exemple : resultats.dicoCharts['localisation_gene']['chart']

In [10]:
# preparation des données
# on limite les données à la période de mesures et aux déclaration qui ne couvrent pas l'ensemble de la période
testChartDecla = filtrerDeclas(resultats)

In [21]:
len(testChartDecla), len(filtrerDeclas(resultats, dureeMaxiDeclaEnHeure=48))

(281, 283)

In [11]:
# Analyse visuelle des évolution de déclaration par jour
# Nombre de participants par jour
dfToutTemps = passageDeclas6min(testChartDecla)

## Recherche d'un indicateur global à comparer avec les niveaux de bruit
L'objectif est de trouver un indicateur qui permette de mettre en relation les niveaux de bruit mesurés et les déclarations. Pour pouvoir comparer avec les niveaux de bruit on va s'appuyer sur l'affectation des participants à un point fixe dans la Bdd, table ressenti.participants

In [12]:
# pour les graphs plus loin
slider = alt.binding_range(min=80, max=109, step=1)
select_day = alt.selection_single(
    name="date_gene", fields=["date_gene"], bind=slider, init={"date_gene": 80}
)

In [13]:
# Agrégation des données de façon à n'avoir qu'une seule ligne par par de 6 minutes
# on va calculer les indicateurs statistiques classiques : écart_type min, max, moyenne, nombre de vote
# on ne pernd pas la médiane, car les votes "extrême" reste des informatiosn à ne pas considérer comme déviantes
dfToutTempsIndics = grouperDeclaParHorodate(dfToutTemps)

In [14]:
# réprésentation graphiques de l'évolution de la moyenne, des min, max et du nombre de déclaration
graphEvolutionMoyMinMaxStd(dfToutTempsIndics)

alt.LayerChart(...)

### Analyse de l'évolution journalière des déclarations de gêne 1 
analyse simple basée sur la somme de toute les notes
$\sum \limits _{n = 0} ^{nbNote} note$

In [22]:
# exemple
slider = alt.binding_range(min=80, max=109, step=1)
select_day = alt.selection_single(
    name="date_gene", fields=["date_gene"], bind=slider, init={"date_gene": 80}
)
creerChartDeclaHeureMinute(dfToutTemps, hauteur=200).add_selection(
    select_day
).transform_filter(select_day)

alt.LayerChart(...)

In [23]:
# pour info, chart avec tous les jours et la méthode 1
# chart (par période de 6 linute et par jour) des déclarations et de la note cumulée
chart = alt.vconcat()
for i in range(80, 104, 2):
    dfFiltre1 = dfToutTemps.loc[dfToutTemps.date_gene == i]
    dfTempsGroupeNote1 = dfToutTemps.loc[dfToutTemps.date_gene == i]
    dfFiltre2 = dfToutTemps.loc[dfToutTemps.date_gene == i + 1] if i < 107 else None
    dfTempsGroupeNote2 = (
        dfToutTemps.loc[dfToutTemps.date_gene == i + 1] if i < 107 else None
    )
    chart &= (
        (creerChartDeclaHeureMinute(dfFiltre1) | creerChartDeclaHeureMinute(dfFiltre2))
        if i < 107
        else (creerChartDeclaHeureMinute(dfFiltre1, dfTempsGroupeNote1))
    )
chart

alt.VConcatChart(...)

### Analyse de l'évolution journalière des déclarations de gêne 2
Méthode associant à la méthode précédente un coefficient lié à la variabilité des déclaration. Cette variabilité est traduite par un inverse de l'écart type par rapport à une variabilité jugée suffisante, et fixée à 1,5.$$\frac{1.5}{\sigma} * \sum \limits _{n = 0} ^{nbNote} note$$ Cela ne fonctionne que si $\sigma$ est différents de 0.<br>Dans le cas contraire, cette formule s'écrit $$\frac{1.5}{1.5-(NbVal*0.1)} * \sum \limits _{n = 0} ^{nbNote} note$$

In [30]:
# création de l'attribut
dfMoyennePonderee = ajouterIndicateursTest(dfToutTempsIndics)
# visu
domainMax = 150
(
    alt.Chart(dfMoyennePonderee)
    .transform_fold(["moyenne", "note_pondere_std"])
    .mark_line()
    .encode(
        x=alt.X("date_heure:T"),
        y=alt.Y("value:Q", scale=alt.Scale(domain=(0, domainMax))),
        color="key:N",
    )
).add_selection(select_day).transform_filter(select_day)

alt.Chart(...)

### Comparaison des méthodes 1 et 2 

In [25]:
# ajout de l'attribut méthode 1 dans le df de la méthode 2
# fourni via un graph horizontal
(
    creerChartDeclaHeureMinute(dfToutTemps, hauteur=300)
    .add_selection(select_day)
    .transform_filter(select_day)
    | alt.Chart(dfMoyennePonderee)
    .transform_fold(["moyenne", "note_cumul", "note_pondere_std"])
    .mark_line()
    .encode(
        x=alt.X("date_heure:T"),
        y=alt.Y("value:Q", scale=alt.Scale(domain=(0, domainMax))),
        color="key:N",
    )
    .add_selection(select_day)
    .transform_filter(select_day)
).resolve_legend("independent")

alt.HConcatChart(...)

<div class="alert alert-block alert-info">
La méthode de cumul des note produit un rendu fidèle à la fois du nombre de déclarant et de la valeur des notes. Cependant, cela ne fournit pas d'information sur la dispersion des valeurs ou le nombre de votant à un instant précis. De plus, l'échelle de lecture n'est pas commune avec le système de note.  
La méthode de note pondérée présente le même défaut d'échelle par rapport au note, mais apporte un éclairage sur la dispersion (exemple le jour 80), ou le niveau pondéré passe sous le niveau cumulé, du fait de la présence d'une note de faible valeur. Cependant, cela crée des aussides effets de bords lorsque des notes élevées s'arretent. En effet, dans ce cas la dispersion diminue, donc la pondération augmente ; par exemple le jour 85, à partir de 19h, certaines augmentations correspondent à des déclarartions de niveaux élevées qui s'arretent
</div>

### Analyse de l'évolution journalière des déclarations de gêne 3
Méthode basée sur une moyenne pondérée( $ \frac{\sum\limits _{n = 0} ^{10} note * NbApparitionDeLaNote}{\sum\limits _{n = 0} ^{10} NbApparitionDeLaNote}  $) .<br> En L'ajout d'une fonction du nombre de votant pour cette note par rapport à un seuil arbitraire permettant de tester si la masse de votant fourni une inforation supplémentaire :
$\frac{\sum\limits _{n = 0} ^{10} (note + \ln(\frac{NbApparitionDeLaNote}{x})) * (NbApparitionDeLaNote^x)}{\sum\limits _{n = 0} ^{10} NbApparitionDeLaNote^x} $ avec $x =$ Nombre de votant considéré comme représentatifs .<br> l'utilisation du logarithme du $\frac{NbApparitionDeLaNote}{nbVotantConsidéréCommeReprésentatifs}$ permet de créer une notation moins importante que la note fournie pour les notes avec peu de déclarant, et une note plus importante pour les notes fournies avec bcp de déclarants

In [36]:
(
    creerChartDeclaHeureMinute(dfToutTemps, hauteur=300)
    .add_selection(select_day)
    .transform_filter(select_day)
    &
    graphMoyennesPonderees(dfMoyennePonderee, listAttributs=["moyenne", "moy_pond", "moy_pond_sur_k"]).add_selection(select_day).transform_filter(select_day)
)

alt.VConcatChart(...)

Les moyennes pondérées fournissent un indicateur à l'échelle des notes. Elles permettent de donner plus de poids aux notes les plus votées. la différenciation entre des périodes ayant une unique note fournie par n personnes, et les périodes ayant la même unique note, fournie par n + x personnes est bien traitée apr l'ajout du terme correctif $\ln(\frac{NbApparitionDeLaNote}{x})$. Par exemple, dans le cas où 2 périodes présente une moyenne de 10 (note maximale), l'une avec un seul vote, l'autre avec 5 votes, en fixant une note comme représentative à partir de plus de 2 votes, alors la pondération avant moyenne va permettre de transformer les valeurs 10 en $10 + \ln(\frac{1}{2})$ et $10 + \ln(\frac{5}{2})$, soit 9,3 et 10,9 : dans ce cas, la pondération directe des notes permet de différencier les moyennes de fces deux situations.

In [27]:
dfMoyennePonderee = ajouterMoyennePonderee(dfToutTempsIndics)

In [29]:
dfMoyennePonderee

,date_heure,ecart_type,valeurs,moyenne,nb_val,nb_occ,note_min,note_max,var,date_gene,moy_pond_2Votants
0,2022-03-21 06:00:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
1,2022-03-21 06:06:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
2,2022-03-21 06:12:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
3,2022-03-21 06:18:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
4,2022-03-21 06:24:00,0.0,"(10, 10)",10.0,2,{10: 2},10,10,0.0,80,10.000000
...,...,...,...,...,...,...,...,...,...,...,...
5575,2022-04-17 22:36:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853
5576,2022-04-17 22:42:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853
5577,2022-04-17 22:48:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853
5578,2022-04-17 22:54:00,NaN,"(7,)",7.0,1,{7: 1},7,7,NaN,107,6.306853


## TFSD
L'indicateur TFSD est issu de l'étude Grafic de l'UGE et l'université du 92 (ref ?). Il permet de quantifier la présence de voix humaines et cri d'oiseau pendant une période, à partir de fichier wav.
L'extraction des donnée par pas de 6 minutes a été réalisée via R, dans le [notebook suivant](https://github.com/nantodevison/Ech24/blob/main/notebooks/R_seewave.ipynb)

In [ ]:
# Pour information : ouverture, agregation des données et transfert en Bdd
dossier = r"C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\TFSD"
listFichier = []
with os.scandir(dossier) as it:
    for entry in it:
        if entry.name.endswith('.csv') and entry.is_file():
            listFichier.append(os.path.join(dossier, entry.name))
dfTfsd = pd.concat([pd.read_csv(f) for f in listFichier])

In [29]:
alt.Chart(dfTfsd.loc[dfTfsd.site == 'JulesLadoumegue']).transform_density(
    'tsfd',
    as_=['tsfd', 'density'],
).mark_area().encode(
    x="tsfd:Q",
    y='density:Q',
)

alt.Chart(...)

In [22]:
dfTfsd.apply(lambda x: pd.to_datetime())

,Unnamed: 0,periode,tsfd,site,fichier
0,1,1_360,0.04,JulesLadoumegue,20220321_120033_123033
1,2,361_720,0.04,JulesLadoumegue,20220321_120033_123033
2,3,721_1080,0.03,JulesLadoumegue,20220321_120033_123033
3,4,1081_1440,0.04,JulesLadoumegue,20220321_120033_123033
4,5,1441_1800,0.04,JulesLadoumegue,20220321_120033_123033
...,...,...,...,...,...
2,3,721_1080,0.03,Villon,20220419_000033_003033
3,4,1081_1440,0.02,Villon,20220419_000033_003033
4,5,1441_1800,0.02,Villon,20220419_000033_003033
0,1,1_360,0.02,Villon,20220419_003033_004608
